# GetYourGuide Data Science Take Home Test

Author: Kai Chen

Date: Mar 2018

train.csv and prediction.csv contain
- Date
- Keyword_ID
- Ad_group_ID
- Campaign_ID
- Account_ID
- Device_ID
- Match_type_ID

train.csv contains additional information
- Revenue
- Clicks
- Conversions

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
import xgboost
from sklearn.metrics import explained_variance_score

In [22]:
train_df = pd.read_csv('ds_dp_assessment/train.csv')
display(train_df.head())

,Date,Keyword_ID,Ad_group_ID,Campaign_ID,Account_ID,Device_ID,Match_type_ID,Revenue,Clicks,Conversions
0,2014-12-14,88883763407,1042143654876,71672332180,861287123742,298643508640,95725474456,0.0,2,0
1,2014-12-14,837336088038,1038839743255,685307833020,654870334100,298643508640,95725474456,0.0,2,0
2,2014-12-14,220115136780,691864063258,551279817203,212779990172,848779586902,95725474456,0.0,2,0
3,2014-12-14,423389776473,1044670215658,551279817203,212779990172,848779586902,95725474456,0.0,5,0
4,2014-12-14,546746671314,355072204358,919168440848,212779990172,298643508640,95725474456,0.0,2,0


In [24]:
display(train_df.describe())

,Keyword_ID,Ad_group_ID,Campaign_ID,Account_ID,Device_ID,Match_type_ID,Revenue,Clicks,Conversions
count,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06
mean,5.492429e+11,5.514791e+11,5.175203e+11,4.846436e+11,6.273406e+11,5.287287e+11,1.782111e+02,3.385137e+00,4.980832e-02
std,3.166589e+11,3.170199e+11,3.136106e+11,2.207485e+11,3.258393e+11,3.942597e+11,2.327171e+03,6.373065e+00,6.945033e-01
min,9.502900e+05,2.179346e+06,5.045747e+08,1.516649e+11,2.986435e+11,9.572547e+10,0.000000e+00,2.000000e+00,0.000000e+00
25%,2.738680e+11,2.764484e+11,2.463512e+11,2.213542e+11,2.986435e+11,9.572547e+10,0.000000e+00,2.000000e+00,0.000000e+00
50%,5.486864e+11,5.513241e+11,5.178672e+11,6.021828e+11,8.487796e+11,8.725446e+11,0.000000e+00,2.000000e+00,0.000000e+00
75%,8.217763e+11,8.263512e+11,7.850502e+11,6.049053e+11,8.487796e+11,8.944136e+11,0.000000e+00,2.000000e+00,0.000000e+00
max,1.099511e+12,1.099508e+12,1.099401e+12,9.814537e+11,1.077719e+12,8.944136e+11,1.029647e+06,1.827000e+03,1.180000e+03


In [25]:
display(train_df['Date'].unique())

array(['2014-12-14', '2014-12-15', '2014-12-16', '2014-12-17',
       '2014-12-18', '2014-12-19', '2014-12-20', '2014-12-21',
       '2014-12-22', '2014-12-23', '2014-12-24', '2014-12-25',
       '2014-12-26', '2014-12-27', '2014-12-28', '2014-12-29',
       '2014-12-30', '2014-12-31', '2015-01-01', '2015-01-02',
       '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06',
       '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10',
       '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14',
       '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18',
       '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22',
       '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26',
       '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30',
       '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03',
       '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07',
       '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11',
       '2015-02-12', '2015-02-13', '2015-02-14', '2015-

In [26]:
train_df['RPC'] = train_df['Revenue']/train_df['Clicks']
display(train_df.describe())

,Keyword_ID,Ad_group_ID,Campaign_ID,Account_ID,Device_ID,Match_type_ID,Revenue,Clicks,Conversions,RPC
count,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06
mean,5.492429e+11,5.514791e+11,5.175203e+11,4.846436e+11,6.273406e+11,5.287287e+11,1.782111e+02,3.385137e+00,4.980832e-02,5.113287e+01
std,3.166589e+11,3.170199e+11,3.136106e+11,2.207485e+11,3.258393e+11,3.942597e+11,2.327171e+03,6.373065e+00,6.945033e-01,7.113045e+02
min,9.502900e+05,2.179346e+06,5.045747e+08,1.516649e+11,2.986435e+11,9.572547e+10,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
25%,2.738680e+11,2.764484e+11,2.463512e+11,2.213542e+11,2.986435e+11,9.572547e+10,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
50%,5.486864e+11,5.513241e+11,5.178672e+11,6.021828e+11,8.487796e+11,8.725446e+11,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
75%,8.217763e+11,8.263512e+11,7.850502e+11,6.049053e+11,8.487796e+11,8.944136e+11,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
max,1.099511e+12,1.099508e+12,1.099401e+12,9.814537e+11,1.077719e+12,8.944136e+11,1.029647e+06,1.827000e+03,1.180000e+03,5.148236e+05


In [40]:
X_train = train_df[['Keyword_ID', 'Ad_group_ID', 'Campaign_ID', 'Account_ID', 'Device_ID', 'Match_type_ID']]
Y_train = train_df['RPC']
#display(X_train)
display(len(Y_train.unique()))

30184

## Feature engineering

In [20]:
hasher = FeatureHasher(n_features=10,input_type='string')

/Users/kai/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


## Model training
- xgboost

In [42]:
# Create dataset for xgboost
X_train_sub, X_val, Y_train_sub, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

display(X_train_sub.shape)
display(X_val.shape)

(6628338, 6)

(1657085, 6)

In [45]:
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

In [46]:
xgb.fit(X_train_sub, Y_train_sub)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.08, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.75)

In [52]:
predictions = xgb.predict(X_val)
print(predictions)
print(explained_variance_score(predictions,Y_val))

[ 34.650017 156.68808   26.956276 ... 144.94804   39.68755   33.23196 ]
-260.88722642611555


In [51]:
xgdmat=xgboost.DMatrix(X_train_sub, Y_train_sub)
our_params={'eta':0.1,'seed':0,'subsample':0.8,'colsample_bytree':0.8,'objective':'reg:linear','max_depth':3,'min_child_weight':1}
final_gb=xgboost.train(our_params,xgdmat)
tesdmat=xgboost.DMatrix(X_val)
y_pred=final_gb.predict(tesdmat)
print(y_pred)

[35.081413 54.32881  19.11128  ... 43.60347  32.216434 32.216434]
